# Use mdapy efficiently

## The key point is re-using the neighborlist information.

In [1]:
import mdapy as mp
mp.init()

[Taichi] version 1.4.0, llvm 15.0.1, commit fbe92fd8, win, python 3.8.15
[Taichi] Starting on arch=x64


## Build a FCC lattice with 4,000,000 atoms.

In [2]:
FCC = mp.LatticeMaker(3.615, 'FCC', 100, 100, 100)
FCC.compute()

## Generate a system class.

In [3]:
system = mp.System(pos=FCC.pos, box=FCC.box, boundary=[1, 1, 1])

## Check the system information.

In [4]:
system.data.head()

,id,type,x,y,z
0,1,1,0.0000,0.0000,0.0000
1,2,1,1.8075,1.8075,0.0000
2,3,1,1.8075,0.0000,1.8075
3,4,1,0.0000,1.8075,1.8075
4,5,1,0.0000,0.0000,3.6150


If we want to do a series of analysis, we can re-use the neighbor to save time.

## Direct calculation.

In [5]:
%%time
system.cal_centro_symmetry_parameter(12)
system.cal_ackland_jones_analysis()
system.cal_atomic_entropy()

CPU times: total: 6min 50s
Wall time: 21.6 s


In [6]:
system.data.head()

,id,type,x,y,z,csp,aja,atomic_entropy
0,1,1,0.0000,0.0000,0.0000,3.295230e-26,1,-8.645166
1,2,1,1.8075,1.8075,0.0000,1.098410e-26,1,-8.645166
2,3,1,1.8075,0.0000,1.8075,1.098410e-26,1,-8.645166
3,4,1,0.0000,1.8075,1.8075,1.098410e-26,1,-8.645166
4,5,1,0.0000,0.0000,3.6150,2.196820e-26,1,-8.645166


## Re-use neighborlist information.

In [7]:
%%time
system.build_neighbor(rc=5.0, max_neigh=50) # Obtain the neighbor first, the following calculation can use it.
system.cal_atomic_entropy()
system.cal_ackland_jones_analysis()
system.cal_centro_symmetry_parameter(12)

CPU times: total: 5min 25s
Wall time: 12.9 s


In [8]:
system.data.head()

,id,type,x,y,z,csp,aja,atomic_entropy
0,1,1,0.0000,0.0000,0.0000,3.295230e-26,1,-8.645166
1,2,1,1.8075,1.8075,0.0000,1.098410e-26,1,-8.645166
2,3,1,1.8075,0.0000,1.8075,1.098410e-26,1,-8.645166
3,4,1,0.0000,1.8075,1.8075,1.098410e-26,1,-8.645166
4,5,1,0.0000,0.0000,3.6150,2.196820e-26,1,-8.645166
